In [ ]:
import os
import tarfile

directory = "dep33"

# Fonction pour décompresser les fichiers
def decompress_files(directory):
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".tar.bz2"):
                file_path = os.path.join(subdir, file)
                with tarfile.open(file_path, 'r:bz2') as archive:
                    archive.extractall(path=subdir)

decompress_files(directory)


In [11]:
import os
import subprocess

# Source and destination directories
src_directory = "dep33"
dst_directory = "dep33c"

# Create destination directory if it doesn't exist
if not os.path.exists(dst_directory):
    os.makedirs(dst_directory)

# Walk through the source directory
for subdir, _, files in os.walk(src_directory):
    for file in files:
        if file.endswith(".THF"):  # EDIGÉO files
            # Full path to the source .thf file
            input_file_path = os.path.join(subdir, file)
            
            # Determine the output file path 
            relative_subdir = os.path.relpath(subdir, src_directory)
            output_subdir = os.path.join(dst_directory, relative_subdir)
            base_name = os.path.splitext(file)[0]
            output_file_path = os.path.join(output_subdir, base_name + ".gpkg")

            # If not path, create it
            if not os.path.exists(output_subdir):
                os.makedirs(output_subdir)
            
            # Command to convert EDIGÉO to GeoPackage using ogr2ogr
            cmd = [
                'ogr2ogr',
                '-f', 'GPKG',
                output_file_path,  # Destination file
                input_file_path    # Source file
            ]
            
            # Execute the command
            subprocess.run(cmd)

print("Conversion completed.")


Warning 1: A geometry of type MULTIPOLYGON is inserted into layer SUBDFISC_id of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer TLINE_id of geometry type LINESTRING, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTIPOLYGON is inserted into layer SUBDFISC_id of geometry type POLYGON, which is not normally allowed by the GeoPackage specification

Conversion completed.


In [12]:

import geopandas as gpd
import os

code_dep = ["000", "130", "140", "150", "170", "185", "270", "310", 
    "320", "360", "370", "400", "520", "530", "560", "600", "700"]

directories = [f"dep33c/33{code}" for code in code_dep]
output_file = "agglo_bordeaux.gpkg"

all_gdfs = []

# Parcourir le répertoire pour trouver tous les fichiers .gpkg
for dir in directories:
    for _, _, files in os.walk(dir):
        for file in files:
            if file.endswith(".gpkg"):
                file_path = os.path.join(dir, file)
                
                # Lire le fichier .gpkg
                gdf = gpd.read_file(file_path)
                all_gdfs.append(gdf)

# Concaténer tous les GeoDataFrames
combined_gdf = gpd.pd.concat(all_gdfs, ignore_index=True)

# Sauvegarder le GeoDataFrame combiné dans un nouveau fichier .gpkg
combined_gdf.to_file(output_file, driver="GPKG")

KeyboardInterrupt: 

In [ ]:
import folium
import geopandas as gpd

input_file = "agglo_bordeaux.gpkg"

# Charger un fichier GeoPackage en tant que GeoDataFrame
gdf = gpd.read_file(input_file)

# Créer une carte centrée sur Bordeaux
m = folium.Map(location=[44.8378, -0.5792], zoom_start=12, tiles=None)

# Ajouter les tuiles Google Maps à la carte
folium.TileLayer('https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', 
                 attr='Google', name='Google Maps', overlay=True, control=True).add_to(m)

# Convertir le GeoDataFrame en GeoJSON et l'ajouter à la carte
folium.GeoJson(gdf).add_to(m)